In [ ]:
import (
	"fmt"
	"os"
	"time"

	"github.com/mrazza/gonav"
	"encoding/csv"
	"io"
	"strconv"
	"strings"
	"image"
	"image/draw"
	"image/jpeg"
	r2 "github.com/golang/geo/r2"
	metadata "github.com/markus-wa/demoinfocs-golang/v2/pkg/demoinfocs/metadata"
    ex "github.com/markus-wa/demoinfocs-golang/v2/examples"
	demoinfocs "github.com/markus-wa/demoinfocs-golang/v2/pkg/demoinfocs"
	events "github.com/markus-wa/demoinfocs-golang/v2/pkg/demoinfocs/events"
    heatmap "github.com/dustin/go-heatmap"
    schemes "github.com/dustin/go-heatmap/schemes"
	"bytes"
    "reflect"



)

In [ ]:
func checkError(err error) {
	if err != nil {
		panic(err)
	}
}

In [ ]:
print(ok == nil && ok1 == nil && ok2 == nil && ok3 == nil && ok4 == nil && ok5 == nil && ok6 == nil)

In [ ]:
func createImage(points[] r2.Point, mapname string) *image.RGBA {
    if len(points) == 0 {
        return nil
    }
    r2Bounds := r2.RectFromPoints(points...)
    bounds := image.Rectangle{
         Min: image.Point{X: int(r2Bounds.X.Lo), Y: int(r2Bounds.Y.Lo)},
        Max: image.Point{X: int(r2Bounds.X.Hi), Y: int(r2Bounds.Y.Hi)},
    }
    
    var data []heatmap.DataPoint
    for _, p := range points[1:] {
        // Invert Y since go-heatmap expects data to be ordered from bottom to top
        data = append(data, heatmap.P(p.X, p.Y*-1))
    }
    fMap, err := os.Open(fmt.Sprintf("./csgo/data/map/%s.jpg", mapname))
    checkError(err)
    
    imgMap, _, err := image.Decode(fMap)
    checkError(err)
    
    img := image.NewRGBA(imgMap.Bounds())
    draw.Draw(img, imgMap.Bounds(), imgMap, image.ZP, draw.Over)
    
    imgHeatmap := heatmap.Heatmap(image.Rect(0, 0, bounds.Dx(), bounds.Dy()), data, 45, 128, schemes.AlphaFire)
    draw.Draw(img, bounds, imgHeatmap, image.ZP, draw.Over)
    
    return img

}

In [ ]:
func convertPathsToArraysOfPoints(arr[] string) ([]r2.Point, []r2.Point, []r2.Point, []r2.Point, []r2.Point, []r2.Point, []r2.Point, ) {
    var miragePoints []r2.Point
    var dustPoints []r2.Point
    var infernoPoints []r2.Point
    var nukePoints []r2.Point
    var overpassPoints []r2.Point
    var trainPoints []r2.Point
    var vertigoPoints []r2.Point
    for i, x := range arr {
        file, err := os.Open(x)
        checkError(err)
        
        p := demoinfocs.NewParser(file)
        
        header, err := p.ParseHeader()
        checkError(err)
        
        mapMetadata := metadata.MapNameToMap[header.MapName]
        
        if(header.MapName == "de_dust2"){
            var points []r2.Point   
            p.RegisterEventHandler(func(e events.Kill) {
            // Translate positions from in-game coordinates to radar overview image pixels
            if(e.Killer.String() == "sh1ro" && e.Killer.Team == 2) {
                x, y := mapMetadata.TranslateScale(e.Killer.Position().X, e.Killer.Position().Y)
                dustPoints = append(dustPoints, r2.Point{X: x, Y: y})
            }
            })           
        } else if (header.MapName == "de_mirage") { 
            p.RegisterEventHandler(func(e events.Kill) {
            // Translate positions from in-game coordinates to radar overview image pixels
            if(e.Killer.String() == "sh1ro" && e.Killer.Team == 2) {
                x, y := mapMetadata.TranslateScale(e.Killer.Position().X, e.Killer.Position().Y)
                miragePoints = append(miragePoints, r2.Point{X: x, Y: y})
            }
            })                
        } else if (header.MapName == "de_inferno") {  
            p.RegisterEventHandler(func(e events.Kill) {
            // Translate positions from in-game coordinates to radar overview image pixels
            if(e.Killer.String() == "sh1ro" && e.Killer.Team == 2) {
                x, y := mapMetadata.TranslateScale(e.Killer.Position().X, e.Killer.Position().Y)
                infernoPoints = append(infernoPoints, r2.Point{X: x, Y: y})
            }
            })                
        } else if (header.MapName == "de_vertigo") {
            p.RegisterEventHandler(func(e events.Kill) {
            // Translate positions from in-game coordinates to radar overview image pixels
            if(e.Killer.String() == "sh1ro" && e.Killer.Team == 2) {
                x, y := mapMetadata.TranslateScale(e.Killer.Position().X, e.Killer.Position().Y)
                vertigoPoints = append(vertigoPoints, r2.Point{X: x, Y: y})
            }
            })                
        }  else if (header.MapName == "de_train") {
            p.RegisterEventHandler(func(e events.Kill) {
            // Translate positions from in-game coordinates to radar overview image pixels
            if(e.Killer.String() == "sh1ro" && e.Killer.Team == 2) {
                x, y := mapMetadata.TranslateScale(e.Killer.Position().X, e.Killer.Position().Y)
                trainPoints = append(trainPoints, r2.Point{X: x, Y: y})
            }
            })                
        } else if (header.MapName == "de_overpass") {
            p.RegisterEventHandler(func(e events.Kill) {
            // Translate positions from in-game coordinates to radar overview image pixels
            if(e.Killer.String() == "sh1ro" && e.Killer.Team == 2) {
                x, y := mapMetadata.TranslateScale(e.Killer.Position().X, e.Killer.Position().Y)
                overpassPoints = append(overpassPoints, r2.Point{X: x, Y: y})
            }
            })                
        } else if (header.MapName == "de_nuke") {
            p.RegisterEventHandler(func(e events.Kill) {
            // Translate positions from in-game coordinates to radar overview image pixels
            if(e.Killer.String() == "sh1ro" && e.Killer.Team == 2) {
                x, y := mapMetadata.TranslateScale(e.Killer.Position().X, e.Killer.Position().Y)
                nukePoints = append(nukePoints, r2.Point{X: x, Y: y})
            }
            })                
        }
        
        err = p.ParseToEnd()
        checkError(err)    
    }
    return miragePoints, dustPoints, infernoPoints, vertigoPoints, trainPoints, overpassPoints, nukePoints
}

In [ ]:
s1mple_map_paths := []string{ "./demos/blastSpring/blast_spring/BLAST-Premier-Spring-Showdown-2021-gambit-vs-furia-bo3/gambit-vs-furia-m1-mirage.dem", "./demos/blastSpring/blast_spring/BLAST-Premier-Spring-Showdown-2021-gambit-vs-furia-bo3/gambit-vs-furia-m2-overpass.dem", "./demos/blastSpring/blast_spring/BLAST-Premier-Spring-Showdown-2021-gambit-vs-furia-bo3/gambit-vs-furia-m3-train.dem", "./demos/blastSpring/blast_spring/BLAST-Premier-Spring-Showdown-2021-gambit-vs-nasr-bo3/gambit-vs-nasr-m1-dust2.dem", "./demos/blastSpring/blast_spring/BLAST-Premier-Spring-Showdown-2021-gambit-vs-nasr-bo3/gambit-vs-nasr-m2-vertigo.dem", "./demos/blastSpring/blast_spring/BLAST-Premier-Spring-Showdown-2021-heroic-vs-gambit-bo3/heroic-vs-gambit-m1-inferno.dem", "./demos/blastSpring/blast_spring/BLAST-Premier-Spring-Showdown-2021-heroic-vs-gambit-bo3/heroic-vs-gambit-m2-vertigo.dem", "./demos/blastSpring/blast_spring/BLAST-Premier-Spring-Showdown-2021-heroic-vs-gambit-bo3/heroic-vs-gambit-m3-train.dem",}

In [ ]:
miragePoints, dustPoints, infernoPoints, vertigoPoints, trainPoints, overpassPoints, nukePoints := convertPathsToArraysOfPoints(s1mple_map_paths)

In [ ]:
mirageImg := createImage(miragePoints, "de_mirage")
dustImg := createImage(dustPoints, "de_dust2")
infernoImg := createImage(infernoPoints, "de_inferno")
vertigoImg := createImage(vertigoPoints, "de_vertigo")
overpassImg := createImage(overpassPoints, "de_overpass")
nukeImg := createImage(nukePoints, "de_nuke")
trainImg := createImage(trainPoints, "de_train")

In [ ]:
miragePoints

In [ ]:
if(mirageImg == nil){
   os.Exit(1)
}
mirageImg

In [ ]:
if(infernoImg == nil){
    os.Exit(1)
}
infernoImg

In [ ]:
if(dustImg == nil){
    os.Exit(1)
}
dustImg

In [ ]:
if(overpassImg == nil){
    os.Exit(1)
}
overpassImg

In [ ]:
if(trainImg == nil){
    os.Exit(1)
}
trainImg

In [ ]:
if(nukeImg == nil){
    os.Exit(1)
}
nukeImg

In [ ]:
// && e.Killer.Team == 3 (killer.team == 3 means ct), killer.team == 2 means t 